Load API Keys

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

Load Contents from HLB Fixed Deposite Pages

In [3]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader([
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit.html?icp=hlb-en-all-footer-txt-fd",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/fixed-deposit-account.html", 
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/e-fixed-deposit.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/flexi-fd.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/senior-savers-flexi-fd.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/junior-fixed-deposit.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/foreign-fixed-deposit-account.html",
  "https://www.hlb.com.my/en/3personal-banking/help-support/fees-and-charges/deposits.html",
  "https://www.hlb.com.my/en/global-markets/forex-rates.html?icp=hlbglo-en-all-menu-txt-forexrates"
])
data = loader.load()
data

[Document(page_content="\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFixed Deposit - HLB\n\n\n\n\n\n\n\n\n\n\nFixed Deposit - HLB\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPERSONAL\n\n\nBUSINESS\n\n\n\n                            Priority\n\n\nPriority\nIslamic\nGlobal Markets\nBlue Header\nInterim Homepage\nPriority Banking Form\ntest\ntest\nLoan Cal\nLog Viewer\nasdsadsad\nstand alone page\nMaster Form Page\nMaster Form Page\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nsingapore\n\n\n\nhong kong\n\n\n\nvietnam\n\n\n\ncambodia\n\n\n\n\n\n                            ENG\n\n\nLAN A\nLAN B\nLAN C\n\n\n\n\n                            Online Banking\n                        \n\nPersonal\nBusiness\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                About Us\n                            \n\nOur Story\nNews & Updates\nInvestor Relations\nCSR\nAwards\nManagement & BODs\nCareer\n\n\n\n\n                                Discover Our Products\n      

Split the Loaded Data into Chunks

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(data)
texts

[Document(page_content='Fixed Deposit - HLB\n\n\n\n\n\n\n\n\n\n\nFixed Deposit - HLB\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPERSONAL\n\n\nBUSINESS\n\n\n\n                            Priority\n\n\nPriority\nIslamic\nGlobal Markets\nBlue Header\nInterim Homepage\nPriority Banking Form\ntest\ntest\nLoan Cal\nLog Viewer\nasdsadsad\nstand alone page\nMaster Form Page\nMaster Form Page\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nsingapore\n\n\n\nhong kong\n\n\n\nvietnam\n\n\n\ncambodia\n\n\n\n\n\n                            ENG\n\n\nLAN A\nLAN B\nLAN C\n\n\n\n\n                            Online Banking\n                        \n\nPersonal\nBusiness\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                About Us\n                            \n\nOur Story\nNews & Updates\nInvestor Relations\nCSR\nAwards\nManagement & BODs\nCareer\n\n\n\n\n                                Discover Our Products\n                            \n\nDeposits\nCredit Card\nLoans\nInsurance\nInvestment\nRemittance\

Initialize Pinecone

In [5]:
import pinecone

pinecone.init(
  api_key=os.getenv("PINECONE_API_KEY"),
  environment=os.getenv("PINECONE_ENVIRONMENT")
)
index_name = "main-index"
namespace = "hlb-fixed-deposit-with-agent"
if index_name not in pinecone.list_indexes():
  pinecone.create_index(
    name=index_name,
    metric="cosine",
    dimension=1536
  )
index = pinecone.Index(index_name)
index.delete(delete_all=True, namespace=namespace)

d:\Code\langchain-test\venv\Lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


{}

Create Embeddings and Vectors to Insert to Pinecone Database

In [9]:
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
vector = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name, namespace=namespace)

Create Retriever for Vector Search

In [10]:
retriever = vector.as_retriever(search_kwargs={'k': 4})
retriever

VectorStoreRetriever(tags=['Pinecone', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.pinecone.Pinecone object at 0x000001D52379BD90>, search_kwargs={'k': 4})

Creating Prompt Template

In [38]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)
system_message = """
  You are a helpful assistant who should reply to inputs related only to the HongLeong Bank aka HLB.
  You should not answer anything other than provided context.
  Your answer should be detailed but if you have to do calculation, don't show your work or equations, just provide the final value, and you don't need to show reference.
  If you are not sure about deposit type, you can ask for clarification.
  If the bank's name is not provided in the input, answer with the assumption that the question is about HLB.
"""
prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a helpful assistant who should reply to inputs related only to the HongLeong Bank aka HLB."),
  MessagesPlaceholder("chat_history", optional=True),
  ("human", "{input}"),
  MessagesPlaceholder("agent_scratchpad")
])

Creating Chain

In [39]:
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(memory_key="chat_history", input_key="input", k=4)

chain = RetrievalQA.from_chain_type(
  llm=llm,
  chain_type="stuff",
  retriever=retriever,
)

Creating Tool from Chain

In [40]:
from langchain.tools import Tool
from langchain.agents import load_tools

knowledge_base_tool = Tool(
  name="hlb_fixed_deposit_tool",
  func=chain.run,
  description=("This tool is to collect and reterieve information about fixed deposit for HongLeong Bank")
)

tools = [knowledge_base_tool]

Creating Agent

In [41]:
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain.callbacks import get_openai_callback

agent = create_openai_functions_agent(llm=llm, tools=[knowledge_base_tool], prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=[knowledge_base_tool], max_iterations=3, verbose=True)

def ask_question(question):
  with get_openai_callback() as cb: 
    result = agent_executor.invoke({"input": question})
    print(f'{cb}\n')
    print(result['output'])

Running the Chain

In [ ]:
ask_question("What is fixed deposit?")

In [ ]:
ask_question("How many types of fixed deposit are there?")

In [ ]:
ask_question("What are the pros and cons of fixed deposit?")

In [ ]:
ask_question("Who is Jay Z?")

In [ ]:
ask_question("Are you sure you dont know Jay Z? He is a very popular hip hop artist?")

In [ ]:
ask_question("How many types of fixed deposit CIMB provide?")

In [ ]:
ask_question("What are the interest rates for SGD Fixed Deposit?")

In [ ]:
ask_question("Can you find out by browsing the web?")

In [ ]:
ask_question("Lets say I want to invest RM 5000 in SGD Fixed Deposit for 36 months and the interest rate is 5%. Please calculate total withdrawal amount at the end of the term. Take note the interest rate is compund interest?")

In [ ]:
ask_question("Lets say I want to invest RM 5000 in SGD Fixed Deposit for 9 months. Please calculate total withdrawal amount at the end of the term. Take note the interest rate is compund interest.")

In [ ]:
ask_question("Can a non-resident of Malaysia invest in CIMB SGD fixed deposit?")

In [ ]:
ask_question("Who is the CEO of CIMB bank?")

In [ ]:
ask_question("What is CIMB net profile for the last financial year?")

In [ ]:
ask_question("What if Jay Z from United States wants to invest in SGD fixed deposit. Is that ok ?")

In [ ]:
ask_question("What if Michael Jackson wants to invest in SGD Fixed Deposit. Is that ok?")

In [36]:
ask_question("Write a poem for HLB fixed deposit starting with Roses are Red, Violets are blue")



> Entering new AgentExecutor chain...
Roses are red,
Violets are blue,
HLB's fixed deposit,
A safe haven for you.

Golden yields await,
In a garden of trust,
Grow your savings with us,
Investing is a must.

With interest rates so fine,
Your finances will bloom,
Secure your future,
And banish the gloom.

So place your funds,
In HLB's care,
Watch your wealth flourish,
With returns that are fair.

Roses are red,
Violets are blue,
With HLB's fixed deposit,
Your dreams can come true.

> Finished chain.
Tokens Used: 223
	Prompt Tokens: 109
	Completion Tokens: 114
Successful Requests: 1
Total Cost (USD): $0.00451

Roses are red,
Violets are blue,
HLB's fixed deposit,
A safe haven for you.

Golden yields await,
In a garden of trust,
Grow your savings with us,
Investing is a must.

With interest rates so fine,
Your finances will bloom,
Secure your future,
And banish the gloom.

So place your funds,
In HLB's care,
Watch your wealth flourish,
With returns that are fair.

Roses are red,
Violets 

In [45]:
ask_question("how many types of saving accounts are there in a general bank except HLB?")



> Entering new AgentExecutor chain...
As an AI developed to assist with specific queries related to HongLeong Bank (HLB), I can't provide a comprehensive answer about other banks' offerings. However, in general, banks may offer a variety of savings account types, including but not limited to:

1. Basic savings accounts
2. High-yield savings accounts
3. Online savings accounts
4. Money market accounts
5. Student savings accounts
6. Senior savings accounts
7. Children's savings accounts
8. Health savings accounts (HSAs)

The exact types and names of savings accounts can vary significantly from bank to bank, and they often come with different features, interest rates, fees, and balance requirements. For specific information about savings accounts at a particular bank other than HLB, it's best to consult that bank's official website or contact their customer service directly.

> Finished chain.
Tokens Used: 279
	Prompt Tokens: 106
	Completion Tokens: 173
Successful Requests: 1
Total Cost

In [44]:
ask_question("who is obama's bank?")



> Entering new AgentExecutor chain...
I'm sorry, but I cannot provide personal banking information about individuals, including public figures like Barack Obama. If you have any questions related to HongLeong Bank (HLB), feel free to ask, and I'll be happy to assist you with that information.

> Finished chain.
Tokens Used: 151
	Prompt Tokens: 97
	Completion Tokens: 54
Successful Requests: 1
Total Cost (USD): $0.00259

I'm sorry, but I cannot provide personal banking information about individuals, including public figures like Barack Obama. If you have any questions related to HongLeong Bank (HLB), feel free to ask, and I'll be happy to assist you with that information.
